In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

from calitp_data_analysis.tables import tbls

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils

In [2]:
shares = pd.read_parquet('intermediate/draft_ntd_market_shares.parquet')

In [3]:
tracts_and_feeds = gpd.read_parquet('intermediate/feeds_tract_geo.parquet')

In [9]:
tracts_and_feeds.columns

Index(['GEOID', 'geometry', 'feed_key'], dtype='object')

In [16]:
dim_feeds = (tbls.mart_gtfs.dim_schedule_feeds() >> filter(_.key.isin(tracts_and_feeds.feed_key))
             >> select(_.key, _.base64_url) >> rename(feed_key = _.key)
             >> inner_join(_, tbls.mart_transit_database.dim_gtfs_datasets(), on = 'base64_url')
             >> inner_join(_, tbls.mart_transit_database.dim_gtfs_service_data(), on = {'key': 'gtfs_dataset_key'})
             >> select(_.feed_key, _.base64_url, 
             # >> collect()
            )

In [17]:
dim_feeds

,feed_key,base64_url,name,manual_check__link_to_dataset_on_website,manual_check__data_license,manual_check__authentication_acceptable,future_uri,uri,source_record_id,_is_current,...,manual_check__grading_scheme_v1,manual_check__localized_stop_tts,manual_check__stable_url,data_quality_pipeline,key,_valid_from,deprecated_date,type,backdated_regional_feed_type,regional_feed_type
0,0f6a1bef113617f84786d46d49303a4b,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,Spirit Bus Schedule,No,Missing,Yes,None,https://data.trilliumtransit.com/gtfs/monterey...,recy14T38x2kl8n7K,False,...,Unknown,Unknown,Yes - Vendor Website,True,6d1d4573fa4905d7ad736e1bad59785d,2023-03-15 00:00:00+00:00,None,schedule,None,None
1,0f6a1bef113617f84786d46d49303a4b,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,Spirit Bus Schedule,Unknown,Unknown,Yes,None,https://data.trilliumtransit.com/gtfs/monterey...,recy14T38x2kl8n7K,False,...,Unknown,Unknown,Yes,True,61f86b62265214bf83b99574e156afd4,2022-10-26 00:00:00+00:00,None,schedule,None,None
2,0f6a1bef113617f84786d46d49303a4b,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,Spirit Bus Schedule Historic,No,Missing,Yes,None,https://data.trilliumtransit.com/gtfs/monterey...,recy14T38x2kl8n7K,True,...,Unknown,Unknown,Yes - Vendor Website,None,8363a7be8f2519ea5d9c33abeaa153d9,2024-01-06 00:00:00+00:00,2024-01-05,schedule,None,None
3,0f6a1bef113617f84786d46d49303a4b,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,Spirit Bus Schedule,Unknown,Unknown,Yes,None,https://data.trilliumtransit.com/gtfs/monterey...,recy14T38x2kl8n7K,False,...,Unknown,Unknown,Yes - Vendor Website,True,ba500c92bfce3a73bee6262c4c7a6bea,2022-11-22 00:00:00+00:00,None,schedule,None,None
4,294252b3b4b42fbb31a31ce184fcb3f5,aHR0cHM6Ly9pcG9ydGFsLnNhY3J0LmNvbS9HVEZTL1NSVE...,Sacramento Schedule,Unknown,Unknown,Yes,None,https://iportal.sacrt.com/GTFS/SRTD/google_tra...,recbzZQUIdMmFvm1r,False,...,Unknown,Unknown,Yes - Provider Website,True,ae1c2f25d0d162bf2a0e90527a8eecec,2023-01-19 00:00:00+00:00,None,schedule,None,None


In [7]:
service_data = tbls.mart_transit_database.dim_gtfs_service_data() >> filter(_._is_current) >> collect()

In [8]:
service_data >> inner_join(_, tracts_and_feeds, on = 

,key,name,source_record_id,service_key,gtfs_dataset_key,customer_facing,category,fares_v2_status,manual_check__fixed_route_completeness,manual_check__demand_response_completeness,_is_current,_valid_from,_valid_to
0,053b17e1360817ecf1b5a21e16126484,MVGO – MVGO Alerts,recI99XuoZfQb7Lc4,a844f7b4ebfcf2833e87260bf3447bed,088fbeac48abfeda62a9398392675145,False,None,[],None,None,True,2023-12-12 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
1,b124745e67e5c8acd80fc80d1064fd04,B-Line – B-Line Alerts,recHtX7WV7H5gZRwd,d584928c5feaa1accef0859f74a058f8,8cf9b2748fcc798dca5bdad2c3e2b28f,True,None,[],None,None,True,2023-10-26 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
2,2eeb353c5cd9fb78491e95e20c34c571,FlixBus – Flixbus Schedule,recP2TyODJKLnKfoT,9daae6a3c07fbed7638796e10def9b9d,a37760dde6b9fdcb76b82e57afab7274,True,None,[],None,None,True,2023-05-18 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
3,3fac05fca33e1c0f7c947fc09c3a9cf4,Glenn Ride – Glenn Schedule,recah4Ls5Sq2YOMhM,b35a15809e09aa7128945ead98b35bb6,72db364f3248e2eb7940bea6ccab1d72,True,None,[],None,None,True,2023-06-13 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
4,3da3f1c40d682726f6004e7dd435599e,B-Line – B-Line Trip Updates,recOm2XjKnk1PdVdA,d584928c5feaa1accef0859f74a058f8,99c066f0726f0c9cf117d9754121ee18,True,None,[],None,None,True,2023-10-26 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,f69de35714542a5531fcd1db1c7493d6,Angel Island-Tiburon Ferry – Bay Area 511 Ange...,recxXDglEfklpKMlM,643839b344145322d3f98fb19af40557,538cd77b8319780a8bf412668f24440d,False,precursor,[],Unknown,Unknown,True,2023-10-25 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
845,570a2604a28cb0f80cb6cec2a9c1bd2d,"Tahoe Truckee Area Regional Transportation – ""...",recmJSB9leIMYkoGP,e16b334aa0877fa87ff3441bbaad3e93,6fda78099793184fe08dd78945d188c0,True,primary,"[Vendor published, Fares v2 updates in progres...",Unknown,Unknown,True,2023-10-25 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
846,0a39061670ba6d033e0b35d040dd21a2,Fairfield and Suisun Transit – Bay Area 511 Fa...,recJqXfXgmFD5wA5X,2621745b8b20cb8dbf4445049ad14bda,0f5e1b251db53223200c5bfc365d33f2,False,precursor,[],Unknown,Unknown,True,2023-10-26 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
847,235b09a3eabfe58660864a8426e14c54,Antelope Valley Transportation Authority – Ant...,recCXo3GZl7eEYJDL,8d62a5cec6fbb88a455322a60cf8dbef,e681c3a8dafa2c80e5b8e2cdd01f917a,True,primary,[],Complete,Unknown,True,2023-10-25 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00
